In [1]:
import tensorflow as tf
import numpy as np 
import matplotlib.pyplot as plt
import os

/home/ubuntu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from tensorflow.keras.callbacks import TensorBoard,ModelCheckpoint,EarlyStopping,LearningRateScheduler,Callback
from tensorflow.keras import backend as K
import numpy as np
from tensorflow.keras import Model
# Next the model definition
from tensorflow.keras.layers import ZeroPadding2D,Conv2D,Cropping2D,Lambda,Concatenate,BatchNormalization,Activation,Lambda,Input,Reshape,Dropout

In [3]:
import random

In [4]:
#For testing the generator
#import tensorflow as tf
#import tensorflow.contrib.eager as tfe
#tfe.enable_eager_execution()

In [5]:
from semi_supervised_function import *

In [6]:
#run_on_cpu()
def _expand(tensor111, dim1, dim2):
    """
    Expands a tensor to have dimensions dim1 and dim2
    """
    ans = tf.keras.backend.repeat_elements(tensor111, dim1, 1)
    ans = tf.keras.backend.repeat_elements(ans, dim2, 2)
    return ans


In [7]:
#parser with many different modfications
def parser_train(record):
    '''Function to parse a TFRecords example'''
    
    features = tf.parse_single_example(
    record,
    features={
    'height': tf.FixedLenFeature([], tf.int64),
    'width': tf.FixedLenFeature([], tf.int64),
    'image_raw': tf.FixedLenFeature([], tf.string),
    'mask_raw': tf.FixedLenFeature([], tf.string),
    'depth': tf.FixedLenFeature([], tf.float32),
    'is_train': tf.FixedLenFeature([], tf.float32),
    })

    image = tf.decode_raw(features['image_raw'], tf.float64)
    annotation = tf.decode_raw(features['mask_raw'], tf.uint8)

    #height = tf.cast(features['height'], tf.int32)
    #width = tf.cast(features['width'], tf.int32)

    annotation=tf.to_double(annotation)
    annotation = tf.reshape(annotation, (101, 101, 1))
    #,features['is_train']
    depth=tf.reshape(features['depth'],shape=(1,1,1))
    scaling=tf.reshape(features['is_train'],shape=(1,1))
    
    #we can add all sorts of stuff to the image
    image = tf.reshape(image, (101, 101,1))
    
    
    image_=tf.concat(axis=2,values=[image,annotation])
    
    #same flips for image and annotaition
    image_=tf.image.random_flip_left_right(image_)
    
    #I still want to train on the full size images
    if random.randint(0,100) < 50:
        image_=tf.random_crop(image_,size=(50,50,2))
        image_=tf.image.resize_images(images=image_,size=(101,101))
    
    
    
    image_corrupt=image_[:,:,0]
    annotation=image_[:,:,1]
    annotation=tf.cast(annotation,dtype=tf.int8)
    annotation=tf.reshape(annotation, (101, 101, 1))
    image_corrupt=tf.reshape(image_corrupt, (101, 101, 1))
    
    #We make two corrupted versions of the Image. 
    image_corrupt_1=tf.image.random_brightness(image_corrupt,max_delta=0.1)
    #image_corrupt_1=tf.image.random_contrast(image_corrupt_1,lower=0,upper=1)
    image_corrupt_1=tf.image.random_jpeg_quality(image_corrupt_1,min_jpeg_quality=90,max_jpeg_quality=100)
    image_corrupt_1 =tf.clip_by_value(image_corrupt_1, 0.0, 1.0)
    image_corrupt_1=tf.layers.dropout(image_corrupt_1,rate=0.01,training=True)
    #image_corrupt_1=tf.image.grayscale_to_rgb(image_corrupt_1)
    
    image_corrupt_2=tf.image.random_brightness(image_corrupt,max_delta=0.1)
    #image_corrupt_2=tf.image.random_contrast(image_corrupt_2,lower=0,upper=1)
    image_corrupt_2=tf.image.random_jpeg_quality(image_corrupt_2,min_jpeg_quality=90,max_jpeg_quality=100)
    image_corrupt_2 = tf.clip_by_value(image_corrupt_2, 0.0, 1.0)
    image_corrupt_2=tf.layers.dropout(image_corrupt_2,rate=0.01,training=True)
    #image_corrupt_2=tf.image.grayscale_to_rgb(image_corrupt_2)
    
    #mage = tf.reshape(image, (101, 101, 1))
    return {'image_corrupt_1': image_corrupt_1,'image_corrupt_2': image_corrupt_2, 'depth': depth,'scaling':scaling},annotation



In [8]:
#Now we create 3 Datasets from the 3 TF Records

batch_size = 16

#One with both the supervised and the semi supervised
dataset = tf.data.TFRecordDataset(['salty_train.tfrecords','salty_test.tfrecords'])
dataset = dataset.map(parser_train).shuffle(buffer_size=18000*2)
dataset = dataset.batch(batch_size).repeat()

#One Only Supervised train
dataset_sup = tf.data.TFRecordDataset(['salty_train.tfrecords'])
dataset_sup = dataset_sup.map(parser_train).shuffle(buffer_size=5000)
dataset_sup = dataset_sup.batch(batch_size).repeat()

#One with the validation set
valid = tf.data.TFRecordDataset(['salty_valid.tfrecords'])
valid = valid.map(parser).shuffle(buffer_size=5000)
valid = valid.batch(batch_size).repeat()

In [9]:
'''for dic,annotation in tfe.Iterator(dataset):
    break'''

'for dic,annotation in tfe.Iterator(dataset):\n    break'

In [10]:
#case=10

In [11]:
'''plt.imshow(dic['image_corrupt_1'][case,:,:,0])

plt.show()'''

"plt.imshow(dic['image_corrupt_1'][case,:,:,0])\n\nplt.show()"

In [12]:
'''plt.imshow(dic['image_corrupt_2'][case,:,:,0])
plt.show()'''

"plt.imshow(dic['image_corrupt_2'][case,:,:,0])\nplt.show()"

In [13]:
'''plt.imshow(annotation[case,:,:,0])

plt.show()'''

'plt.imshow(annotation[case,:,:,0])\n\nplt.show()'

In [14]:
#We laod the validation set and a small amount of the training set to memory for validation purposes
X_Val,Y_Val=load_to_memory('salty_valid.tfrecords')
X_Val_t,Y_Val_t=load_to_memory('salty_train.tfrecords',size=160)

In [15]:
alpha = K.variable(0)
class NewCallback(Callback):
    def __init__(self, alpha):
        self.alpha = alpha       
    def on_epoch_end(self, epoch, logs={}):
        x= epoch/100
        x= math.exp(-5*(1-x)**2)
        print(x)
        K.set_value(self.alpha, x)

In [19]:
class CustomRegularization(Layer):
    def __init__(self, **kwargs):
        super(CustomRegularization, self).__init__(**kwargs)

    def call(self ,x ,mask=None):
        ld=x[0]
        rd=x[1]
        bce=tf.reduce_mean(tf.keras.losses.mean_squared_error(logits,logits_c),reduction_indices=(1,2))
        loss2 = alpha*K.sum(bce)
        self.add_loss(loss2,x)
        #you can output whatever you need, just update output_shape adequately
        #But this is probably useful
        return bce

    def get_output_shape_for(self, input_shape):
        return (input_shape[0][0],1)


In [20]:
def dice_loss(y_true, y_pred):

    bin_crossentropyloss = tf.keras.losses.binary_crossentropy(y_true,y_pred)
    #the scale dsupervised loss scale is a normal Keras Input it is one if this case has a label 0 if not
    super_loss=tf.reduce_mean(scaling*((1-dice_coef(y_true, y_pred))*0.99+bin_crossentropyloss*0.01),reduction_indices=(1,2))
    
    #We add unsupervised losses at multiple levels: 1. the final representation resulting form deeplab
    #2.) The deepes representation resulting from the deeplabs
    #3.) the final logits. 
    #unsuper_loss_1=1/3*tf.reduce_mean(0.5*tf.keras.losses.mean_squared_error(low_rep,low_rep_c),reduction_indices=(1,2))
    #unsuper_loss_2=1/3*tf.reduce_mean(0.5*tf.keras.losses.mean_squared_error(deep_rep,deep_rep_c),reduction_indices=(1,2))
    #unsuper_loss_3=0.5*tf.reduce_mean(tf.keras.losses.mean_squared_error(logits,logits_c),reduction_indices=(1,2))
    
    #The unsupervised factor has to be determined, in the paper they suggest slowly increasing it during training. 
    return super_loss #+ scale*(unsuper_loss_3)

In [21]:
from DeepLabv3Plus_tf import Deeplabv3

In [22]:
dl1=Deeplabv3(weights='pascal_voc', input_tensor=None, input_shape=(128, 128, 3), classes=21, backbone='xception', OS=16, alpha=1.)

In [23]:
#Two differently augmented images a input
image_corrupt_1 = Input((101, 101, 1),name="image_corrupt_1")
image_corrupt_2 = Input((101, 101, 1),name="image_corrupt_2")

#The scaling input, which is an array of 1 an 0 if a label exists or not 
scaling = Input((1,1), name="scaling" )

#the depth input
depth = Input( (1,1,1),name="depth" )

#A cnn layer w use to go to 3 dimensions
cnn_1=Conv2D(3, (1, 1), activation='relu')

#we pad them 
s = ZeroPadding2D( padding = ((13, 14), (13, 14)) ) (image_corrupt_1)
s_c=ZeroPadding2D( padding = ((13, 14), (13, 14)) ) (image_corrupt_2)


#we get them to 3 dimensions
img_input =  cnn_1(s) 
img_input_c=cnn_1(s_c) 

#Do forward pass with different dropout configurations Im pretty sure this should work out as I had hoped, ( cause of the named stuff)
extract=dl1(img_input)
extract_c=dl1(img_input_c)
extract=Dropout(0.01)(extract)
extract_c=Dropout(0.01)(extract_c)

In [24]:

#We add depth
dd = Lambda(lambda x: x * 0.001) (depth)
dd = Lambda(lambda x: _expand(x, 128, 128) )(dd)

#We define a "get logits" function. 


cn_l=Concatenate(axis=3,name = "con_last")
cnv_l=Conv2D(128,(1,1),activation="relu",name = "conv_last")
cnv_ll=Conv2D(1,(1,1),name = "logits_last",activation="sigmoid")
crop_ll=Cropping2D(cropping=((13, 14), (13, 14)),name = "crop_last" )
def get_logits(extract):
    x_2=cn_l([extract,dd])
    x_2=cnv_l(x_2)
    logits=cnv_ll(x_2)
    logits =  crop_ll(logits)
    return logits

#We ge tthe logits from both
logits=get_logits(extract)
logits_c=get_logits(extract_c)
cr = CustomRegularization()([logits,logits_c])

model=Model(inputs=[image_corrupt_1,image_corrupt_2,depth,scaling],outputs=[logits,cr])

    


In [25]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
image_corrupt_1 (InputLayer)    (None, 101, 101, 1)  0                                            
__________________________________________________________________________________________________
image_corrupt_2 (InputLayer)    (None, 101, 101, 1)  0                                            
__________________________________________________________________________________________________
zero_padding2d_6 (ZeroPadding2D (None, 128, 128, 1)  0           image_corrupt_1[0][0]            
__________________________________________________________________________________________________
zero_padding2d_7 (ZeroPadding2D (None, 128, 128, 1)  0           image_corrupt_2[0][0]            
__________________________________________________________________________________________________
conv2d (Co

In [26]:
import math

In [27]:
class IntervalEvaluation(Callback):
    def __init__(self, validation_data=(),training_data=() ,interval=10):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data
        #also load training data
        self.X_Val_T,self.Y_Val_T =training_data
        self.score_list=[0]
        self.score_list_train=[0]

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=1)[0]
            y_pred_t = self.model.predict(self.X_Val_T, verbose=1)[0]
            #first we have to threshhold
            prec_score=return_iou(y_pred,self.y_val)
            prec_score_t=return_iou(y_pred_t,self.Y_Val_T)
            
            print('Validation score is {}.Train score is {} Best so far is {}'.format(prec_score,prec_score_t, max(self.score_list)))
            
            if prec_score > max(self.score_list):
                print('Saving model-tgs-salt-IV.h5')
                self.model.save_weights('model-tgs-salt-IV.h5')
            
            self.score_list.append(prec_score)
            self.score_list_train.append(prec_score_t)
            
            
def _expand(tensor111, dim1, dim2):
    """
    Expands a tensor to have dimensions dim1 and dim2
    """
    ans = tf.keras.backend.repeat_elements(tensor111, dim1, 1)
    ans = tf.keras.backend.repeat_elements(ans, dim2, 2)
    return ans
    

In [28]:
import math
def step_decay(epoch):
    initial_lrate = 0.001 
    drop = 0.95
    epochs_drop = 5.0
    lrate = initial_lrate * math.pow(drop,  
            math.floor((1+epoch)/epochs_drop))
    
    if (lrate < 5e-7):
        lrate = 5e-7
      
    print('Changing learning rate to {}'.format(lrate))
    return lrate

In [29]:
#Callbacks
#1- Tensorboard
tens=TensorBoard(log_dir='../logs/semi_upsampled_mobilenet_ramping_onlly_preds_cropping_correct_loss_high_lr', histogram_freq=0, batch_size=32)
#LR Sched
lrate = LearningRateScheduler(step_decay)
#THe evaluation too. 
internalEval = IntervalEvaluation( validation_data=(X_Val,Y_Val),training_data= (X_Val_t,Y_Val_t),interval = 1 )
#Model Evaluation 
check=ModelCheckpoint(filepath="semi_upsampled_mobilenet_ramping_onlly_preds_cropping_correct_loss_high_lr.h5",save_best_only=True,monitor="val_loss")


In [ ]:
model.compile(optimizer='adam', loss=[dice_loss,None])
#fit for one epoch 
model.fit(dataset,steps_per_epoch=int((8000)/batch_size), epochs=1000,validation_data=valid,validation_steps=20,callbacks=[tens,lrate,check,internalEval,NewCallback(alpha)],initial_epoch=1)#18000+


Epoch 2/1000
Changing learning rate to 0.001
160/160 [==============================] - 5s 32ms/step
Validation score is 0.60625.Train score is 0.62625 Best so far is 0
Saving model-tgs-salt-IV.h5
0.007442860710056644
Epoch 3/1000
Changing learning rate to 0.001
 33/500 [>.............................] - ETA: 14:31 - loss: 0.0237 - crop_last_loss: 0.0229

In [ ]:
for j in range (3,500):
#Get the modifier for the loss 
    x= j/100
    x=math.exp(-5*(1-x)**2)
    print(x)
    scale=tf.convert_to_tensor(x,dtype=tf.float32,name="scale_Loss")
    #recompile loss with new scaling term 
    model.compile(optimizer='adam', loss=[dice_loss,zero_loss])
    #fit for one epoch 
    model.fit(dataset,steps_per_epoch=int((8000)/batch_size), epochs=j+1,validation_data=valid,validation_steps=20,callbacks=[tens,lrate,check,internalEval],initial_epoch=j)#18000+
    

In [ ]:
#In a way at each epoch we "sample" 4k true example and 4k unlabeled examples. 

model.compile(optimizer='adam', loss=[dice_loss])
model.fit(dataset,steps_per_epoch=int((8000)/batch_size), epochs=1000,validation_data=valid,validation_steps=20,callbacks=[tens,lrate,check,internalEval])#18000+